In [12]:
import random
import textwrap
import openpyxl
from reportlab.lib.pagesizes import letter
from reportlab.lib.units import inch
from reportlab.pdfgen import canvas

def create_puzzle_table(words):
    max_len = max(len(word) for word in words) + 10  # Added buffer to increase grid size
    puzzle = [[' ' for _ in range(max_len)] for _ in range(max_len)]
    failed_words = []  # To keep track of words that couldn't be placed
    for word in words:
        direction = random.choice(['horizontal', 'vertical'])
        placed = False
        if direction == 'horizontal':
            placed = place_word_horizontal(word, puzzle)
        else:
            placed = place_word_vertical(word, puzzle)
        if not placed:
            failed_words.append(word)
    fill_random_letters(puzzle)
    if failed_words:
        print(f"Failed to place words: {', '.join(failed_words)}")
    return puzzle


def place_word_horizontal(word, puzzle, max_attempts=5):
    word_len, max_len = len(word), len(puzzle)
    for _ in range(max_attempts):
        row, col = random.randint(0, max_len - 1), random.randint(0, max_len - word_len)
        if all(puzzle[row][col + i] in [' ', word[i]] for i in range(word_len)):
            for i, letter in enumerate(word):
                puzzle[row][col + i] = letter
            return True
    return False

def place_word_vertical(word, puzzle, max_attempts=30):
    word_len, max_len = len(word), len(puzzle)
    for _ in range(max_attempts):
        row, col = random.randint(0, max_len - word_len), random.randint(0, max_len - 1)
        if all(puzzle[row + i][col] in [' ', word[i]] for i in range(word_len)):
            for i, letter in enumerate(word):
                puzzle[row + i][col] = letter
            return True
    return False

def fill_random_letters(puzzle):
    alphabet = 'ABCDEFGHIJKLMNOPQRSTUVWXYZ'
    for i in range(len(puzzle)):
        for j in range(len(puzzle[i])):
            if puzzle[i][j] == ' ':
                puzzle[i][j] = random.choice(alphabet)

def display_puzzle(puzzle, words, tab_name, headers):
    page_width, page_height = 8.5 * inch, 11 * inch
    c = canvas.Canvas(f"{tab_name}.pdf", pagesize=(page_width, page_height))
    usable_width, usable_height = page_width - (0.5 * inch * 2), page_height - (0.5 * inch * 2)
    cell_size = min(usable_width // len(puzzle), usable_height // len(puzzle[0]))
    x_offset = (page_width - cell_size * len(puzzle[0])) // 2
    y_offset = (page_height - cell_size * len(puzzle)) // 2
    x_offset, y_offset = max(x_offset, 0.5 * inch), max(y_offset, 0.5 * inch)

    # Use tab_name as the header
    c.setFont("Helvetica", 12)
    c.drawString(x_offset, page_height - 50, tab_name)  # tab_name is used as the header here

    # Draw the puzzle
    for i, row in enumerate(puzzle):
        for j, letter in enumerate(row):
            x0, y0 = x_offset + j * cell_size, y_offset + i * cell_size
            c.rect(x0, y0, cell_size, cell_size)
            c.setFont("Helvetica", 10)
            c.drawString(x0 + cell_size // 3, y0 + cell_size // 3, letter.upper())

    # Draw words at the bottom
    bottom_text = ", ".join(words)
    max_width = 70
    wrapped_text = textwrap.wrap(bottom_text, width=max_width)
    c.setFont("Helvetica", 10)
    text_y_offset = y_offset - 20
    line_height = 12
    for line in wrapped_text:
        c.drawString(x_offset, text_y_offset, line)
        text_y_offset -= line_height

    # Save the PDF
    c.save()
    print(f"PDF file for '{tab_name}' saved successfully.")


def generate_pdf_from_excel(input_file):
    wb = openpyxl.load_workbook(input_file)
    for sheet_name in wb.sheetnames:
        sheet = wb[sheet_name]

        # Extract headers from the first row, assuming the first row contains headers
        headers = [cell.value for cell in sheet[1]]

        # Extract data from the rest of the sheet, skipping the first row
        # Since values_only=True, you directly get cell values, not cell objects, so no need for .value
        data = [str(cell) for row in sheet.iter_rows(min_row=2, values_only=True) for cell in row if cell is not None]

        puzzle = create_puzzle_table(data)
        display_puzzle(puzzle, data, sheet_name, headers)

def main():
    input_file = 'Chemistryteks_5.xlsx'
    generate_pdf_from_excel(input_file)

if __name__ == "__main__":
    main()


Failed to place words: chemicalfamily, ionizationenergy
PDF file for 'periodic table' saved successfully.
Failed to place words: energylevel
PDF file for 'Atomic theory' saved successfully.
Failed to place words: subscript
PDF file for 'Nomenclature' saved successfully.
PDF file for 'Chemical bonding' saved successfully.
PDF file for 'Mole concept' saved successfully.
Failed to place words: stoichiometry
PDF file for 'Chemical equations and reaction' saved successfully.
PDF file for 'Gas behavior' saved successfully.
Failed to place words: hydrogenbonding, surfacearea
PDF file for 'Behaviors of solution' saved successfully.
PDF file for 'acids and bases' saved successfully.
Failed to place words: lawofconservationofenergy
PDF file for 'Energy in chemical reactions' saved successfully.
PDF file for 'Nuclear chemistry' saved successfully.
